In [1]:
from langchain.llms import OpenAI
from langchain_community.chat_models.openai import ChatOpenAI
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain.chains import SequentialChain
from langchain.callbacks import get_openai_callback
import os
import json
import pandas as pd
from dotenv import load_dotenv
import PyPDF2

load_dotenv()

False

In [16]:
file_path=r'D:\ProjectsAI\flashcard-generator\data\test.txt'

with open(file_path, 'r') as f:
    text=f.read()

In [17]:
llm=ChatOpenAI(openai_api_key=os.environ['OPENAI_API_KEY'],model_name='gpt-3.5-turbo',temperature=0.5)

In [18]:
prompt_template="""
Given Text: {text}

As an expert in creating educational flashcards, your task is to distill the essence of the provided text into a concise and informative flashcard.
This flashcard should facilitate learning by presenting a objective question that directly relates to the core information in the text, followed by a precise answer. 
Your goal is to aid in the retention of this knowledge through effective question and answer framing.

Format your response according to the JSON structure provided below. Ensure your submission includes both a 'question' that encapsulates a key point or detail from the text
and an 'answer' that accurately and succinctly responds to the question posed.

RESPONSE_JSON_STRUCTURE = {
    "question": "What is the main point or a key detail of the provided text?",
    "answer": "The main point or key detail of the text is..."
}

Note: Replace the placeholders in the response structure with your actual question and answer derived from the text.
"""

In [19]:
card_generation_prompt=PromptTemplate(
    input_variables=['text'],
    template=prompt_template
)

card_chain=LLMChain(llm=llm,prompt=card_generation_prompt, output_key='card',verbose=True)

In [14]:
response=card_chain(text)
print(response)



> Entering new LLMChain chain...


ValueError: Missing some input keys: {'text'}